<h1 style="margin-bottom: 0px; font-size: 36px">Stimulus Displayer</h1>
<h3 style="margin-top: 0px;font-size: 25px">and Setup Guide</h3>

This notebook allows you to read and verify whether your stimulus is being displayed properly. It checks if the sequence settings are correct, and enables the creation of GIFs to visualize the stimulus output.

### File and Folder Setup 

Before running the notebook, ensure the following files and folders are properly set up in your working directory:

- **VEC Folder**: Contains `.vec` files (which describe the sequences of frames).
- **BIN Folder**: Contains `.bin` files (which store the visual frames to be displayed).
- **StimList Folder**: For VDH stimulus setups, include the **phasmasks list** here.
- **MEA Setup**: A setup description is needed to correctly read the `.bin` file (typically provided by the stimulus generation system).
- **Frame Rate**: The frame rate is set based on the GUI maximum (typically about 100Hz, but this can vary depending on your computer's performance).

### Sequence Selection

The notebook allows you to select which stimulus sequences you want to display in the GUI.

- The **VEC file** contains information about each sequence, with each line representing a frame triggered on the DMD (Digital Micromirror Device). The first line of the `.vec` file is a summary, and the next lines follow this structure (left to right column):
  - `(0 : nothing, 1 : switch the next phasemask)` 
  - `(visual frame index in the bin file)`
  - `(reserved for future color settings)`
  - `(0 : laser shutter close, 1 : laser shutter open)`
  - `(sequence number with repetition number added to it)`

### Steps for Using the Notebook
1. **Enter parameters**: You will first have to enter to right bin, vec, phasemask file, mea and frame rate
2. **Select Sequences**: You can choose specific sequences to display in the GUI (one or more sequences number separatd with space or ,), or display all available sequences (empty input).
3. **Display the GUI**: After selecting your sequences, the GUI will display the stimuli. During the display, the last 10 seconds of the stimulus will be captured and saved as frames. If while playing the visual stimulus, the vec calls for a non existing frame in the bin, an error message is displayed and the unknown frame is replace by the first frame of the bin.
4. **Create a GIF**: The captured frames can be used to create a GIF of the visual stimulus to ensure that the sequence is displayed correctly.
5. **Generate GIF for All Sequences**: After confirming the visual display, you can generate a GIF for the every visual stimulus based on the `.vec` file sequences, one gif per sequence.


### Seting up environment

In [ ]:
### import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
from tqdm.auto import tqdm
from binfile import *
import pygame
import imageio

# Create a drifting grating
def find_files(path):
    return sorted([f for f in os.listdir(path) if (os.path.isfile(os.path.join(path, f)) and os.path.splitext(f)[1] in [".vec",".bin",".txt"])])       #If no, the path is considered as a folder and return the name of all the files in alphabetic order
def read_file_to_list(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()  # Reads all lines into a list
    return [line.strip() for line in lines]  # Strip newlines and return as a list



root = r"./" #put "./" to take the current working directory or put the path to you directory of interest
BIN = os.path.join(root,"BIN/")

VEC = os.path.join(root,"VEC/")
StimList = os.path.join(root, "StimList/")
gif_folder = os.path.join(root,"GIFs")


### Input stimulus parameters

In [ ]:
#Select Vec file
vec_files = find_files(VEC)
print(*['{} : {}'.format(i,vec_file) for i, vec_file in enumerate(vec_files)], sep="\n")
vec_number = int(input("\nSelect VEC : "))
vec_file = vec_files[vec_number]
print(f"\nSelected recording : {vec_file} \n")

vec_trigs = np.loadtxt(os.path.join(VEC,vec_file))[1:]

print("-------------------------------\n")

#Select Bin File
bin_files = find_files(BIN)
print(*['{} : {}'.format(i,bin_file) for i, bin_file in enumerate(bin_files)], sep="\n")
bin_number = int(input("\nSelect BIN : "))
bin_file = bin_files[bin_number]
print(f"\nSelected recording : {bin_file} \n")

print("-------------------------------")

is_holo = sum(vec_trigs[:,0]) > 1
if is_holo:
    print("This stim includes holographic stimulation !")
    stim_files = find_files(StimList)
    print(*['{} : {}'.format(i,stim) for i, stim in enumerate(stim_files)], sep="\n")
    stim_number = int(input("\nSelect Stim List file (put 999 to skip): "))
    if stim_number == 999:
        is_holo= False
    else:
        stim_list = read_file_to_list(os.path.join(StimList,stim_files[stim_number]))
        print(f"\nSelected recording : {stim_files[stim_number]} \n")


MEA = int(input("\nEnter MEA number 2 or 3 :"))
if MEA==2: 
    threshold  = 150e+3
    pxl_size_dmd = 3.5
    size_dmd = [864, 864]      # dimensions of the DMD, in pixels
if MEA==3:
    threshold  = 170e+3   
    size_dmd = [760, 1020]      # dimensions of the DMD, in pixels
    pxl_size_dmd = 2.5          # The size of one pixel of the DMD in µm? on the camera or in reality?
    
binObj = BinFile(os.path.join(BIN,bin_file), size_dmd[0], size_dmd[1], MEA, mode='r')

print("\n-------------------------------")


FPS = int(input("\nFrame rate : "))


# Color dictionary for the second dot column
color_dict = {
    0: (255, 255, 255),  # white
    1: (255, 0, 0),      # red
    2: (0, 255, 0),      # green
    3: (128, 0, 128),    # violet
    4: (50, 50, 50)      # dark
}

### Select the sequence and display the gui


In [ ]:
print(f'Available stimulus sequences : {np.sort(np.unique(vec_trigs[:, -1] // 100)).astype(int)}', sep="\n")
selected_keys = [int(key) for key in input("\nSelect the keys to display (keep empty for all of them, multiple accepted):\n").replace(',', ' ').split()]

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 50+size_dmd[1]+50+250, size_dmd[0]+100
DOT_RADIUS = 50
TRIANGLE_SIZE = 50
FONT_SIZE = 50

gif_list=[]

# Initialize Pygame window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Information Display")

def load_image(row,binObj):
    try :
        image = binObj.read_frame(int(row[1]))  # Assuming this returns a normalized array between 0 and 1
        image = np.abs(255*(image)).astype(np.uint8)
    except AssertionError:
        print(f"Index out of range for image {int(row[1])} (sequence {int(row[-1])}), displaying first frame ! ")
        image = binObj.read_frame(0)
        image = np.abs(255*(image)).astype(np.uint8)
    return image


# Function to draw a triangle
def draw_triangle(surface, color, position):
    points = [
        (position[0], position[1] + TRIANGLE_SIZE),
        (position[0] - TRIANGLE_SIZE, position[1] - TRIANGLE_SIZE),
        (position[0] + TRIANGLE_SIZE, position[1] - TRIANGLE_SIZE),
    ]
    pygame.draw.polygon(surface, color, points)

#Taking only specified sequences or all sequences
if selected_keys:
    mask = np.array([any(key * 100 <= value < (key + 1) * 100 for key in selected_keys) for value in vec_trigs[:, -1]])
    data = vec_trigs[mask]
    phasemasks = np.cumsum(vec_trigs[:,0])[mask].astype(int)
else:
    data = vec_trigs
    phasemasks = np.cumsum(vec_trigs[:,0]).astype(int)
    
current_index = 0  # Change this to display a different row

# Set up the clock
clock = pygame.time.Clock()
paused = False  # Track the paused state

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:  # Spacebar to pause/resume
                paused = not paused
        
    # Clear the screen
    screen.fill((128, 128, 128))
    if not paused:
        # Draw the current row of data
        row = data[current_index]
        y_offset = 50  # Fixed position since displaying one row
        x_offset = 50+size_dmd[1]+50
        # Draw the image on the left side
        image = load_image(row,binObj)
        # image 2‑D en niveaux de gris (H, W) → RGB (H, W, 3)
        rgb = np.stack((image,)*3, axis=-1) # Convert to 3-channel greyscale

        # Transpose pour passer en (W, H, 3) comme le veut Pygame
        rgb = np.transpose(rgb, (1, 0, 2))          # ou  rgb = rgb.swapaxes(0, 1)
        image = pygame.surfarray.make_surface(rgb)  
        image = pygame.transform.scale(image, (size_dmd[1], size_dmd[0]))  # Scale the image as needed
        screen.blit(image, (50, y_offset))  # Position the image

        # Prepare the text information on the right side
        font = pygame.font.Font(None, FONT_SIZE)
        # Fifth column: text (bottom)
        text_surface_5 = font.render(str(int(row[4]))[:-2], True, (255, 255, 255))
        text_width = text_surface_5.get_width()
        screen.blit(text_surface_5, (x_offset+100 - text_width // 2, y_offset + 50))

        # Triangle (under text)
        triangle_color = (255, 0, 0) if row[3] == 1 else (50, 50, 50)
        draw_triangle(screen, triangle_color, (x_offset+100, y_offset + 200))
        
        if is_holo:
            font = pygame.font.Font(None, FONT_SIZE//3)
            # Text under triangle for phasemask selected
            text_surface_PM = font.render(stim_list[phasemasks[current_index]], True, (200, 200, 200))
            text_width = text_surface_PM.get_width()
            screen.blit(text_surface_PM, (x_offset+100 - text_width // 2, y_offset + 270))

        # Colored dot (under triangle's text)  --> Laser shutter
        dot_color = (0, 255, 0) if row[0] == 1 else (50, 50, 50)
        pygame.draw.circle(screen, dot_color, (x_offset+100, y_offset + 420), DOT_RADIUS)

        # Colored dot (under first dot) --> Color setting
        dot_color_2 = color_dict.get(row[2], (255, 255, 255))
        pygame.draw.circle(screen, dot_color_2, (x_offset+100, y_offset + 620), DOT_RADIUS)

        # Prepare the text information on the right side
        font = pygame.font.Font(None, FONT_SIZE//2)
        # Fifth column: text (bottom)
        text_surface_1 = font.render(str(int(row[1])), True, (200, 200, 200))
        text_width = text_surface_1.get_width()
        screen.blit(text_surface_1, (x_offset-45, y_offset + size_dmd[0]- 15))

        

        # Update the display
        pygame.display.flip()
        
        
        # Move to the next index
        current_index = (current_index + 1) % len(data)

        # Control the speed of the cycle
        clock.tick(FPS)
        gif_frame = pygame.surfarray.array3d(screen)  # Get the pixel array of the screen
        gif_list.append(np.transpose(gif_frame, (1, 0, 2)))
        gif_list=gif_list[-FPS*10:]
# Quit Pygame
pygame.quit()


### Make a gif of the displayed image (last 10s max)

In [ ]:
# Create a GIF from the list of frames
gif_filename = 'gui_output.gif'
imageio.mimsave(gif_filename, gif_list, duration=0.1, loop=0)  # duration in seconds per frame
print(f"GIF saved as {gif_filename}")

### Generate all GIF for all sequences

In [ ]:
rep_number = 0 # The repetition of the sequence that will be used to make the gif (0 means first rep)
os.makedirs(gif_folder, exist_ok=True)

for seq in tqdm(np.sort(np.unique(vec_trigs[:, -1] // 100)).astype(int)):
    gif_frames = []
    mask = np.array([seq * 100 + rep_number == value for value in vec_trigs[:, -1]])
    data = vec_trigs[mask]
    for row in data:
        gif_frames.append(load_image(row,binObj))
    imageio.mimsave(os.path.join(gif_folder,f"Sequence_{seq}.gif"), gif_frames, fps=FPS, loop=0)  # duration in seconds per frame